In [1]:
!pip install nltk
!pip install spacy
!pip install jieba
!pip install transformers
!pip install sentencepiece
!pip install -U synonyms
!python -c "import synonyms"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 74.0 MB/s eta 0:00:00
  Prepari

In [2]:
import nltk
import spacy
from jieba import posseg
import jieba
import re
from collections import Counter
import synonyms
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nlp_en = spacy.load("en_core_web_sm")
nltk.download('wordnet')
from nltk.corpus import wordnet
from transformers import RobertaTokenizer, RobertaModel
import torch
from scipy.spatial.distance import cosine

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer_en2zh = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="en_XX")
tokenizer_zh2en = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="zh_CN")

tokenizer_en = RobertaTokenizer.from_pretrained('roberta-base')
model_roberta = RobertaModel.from_pretrained('roberta-base')

from transformers import BertTokenizer, BertModel
tokenizer_zh = BertTokenizer.from_pretrained('bert-base-chinese')
model_bert = BertModel.from_pretrained('bert-base-chinese')



Building prefix dict from /usr/local/lib/python3.10/dist-packages/synonyms/data/vocab.txt ...
DEBUG:jieba:Building prefix dict from /usr/local/lib/python3.10/dist-packages/synonyms/data/vocab.txt ...
Loading model from cache /tmp/jieba.uc3c121179eb864b10e70ae5fa25047be.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.uc3c121179eb864b10e70ae5fa25047be.cache



 Synonyms: v3.18.0, Project home: https://github.com/chatopera/Synonyms/

 Project Sponsored by Chatopera

  deliver your chatbots with Chatopera Cloud Services --> https://bot.chatopera.com

>> Synonyms load wordseg dict [/usr/local/lib/python3.10/dist-packages/synonyms/data/vocab.txt] ... 


Loading model cost 1.846 seconds.
DEBUG:jieba:Loading model cost 1.846 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


>> Synonyms on loading stopwords [/usr/local/lib/python3.10/dist-packages/synonyms/data/stopwords.txt] ...
>> Synonyms on loading vectors [/usr/local/lib/python3.10/dist-packages/synonyms/data/words.vector.gz] ...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from transformers.models import align
class Literal_translation:

    def __init__(self,path,save_path):
        with open(path) as f:
          text = f.readlines()
        self.en_li = []
        self.zh_li = []
        for i, sent in enumerate(text):
            if i%2 == 0:
                self.en_li.append(sent.strip())
            elif i%2 == 1:
                self.zh_li.append(sent.strip())
        self.save_path = save_path


    def passive_voice_bei(self):
        e = 1
        regex1 = ' (is|are|was|were|be|been)(.{2,8}ed) '
        # if ',' not in en and '，' not in zh and ',' not in zh:
        for en,zh in zip(self.en_li, self.zh_li):
            
            passive_verb = re.findall(regex1,en)
            
            
            if len(passive_verb) == 1 and len(passive_verb[0]) ==2 and '被' in zh:
                zh_tok_li = posseg.cut(zh)
                verb_li = [str(tok).split('/')[0] for tok in zh_tok_li if str(tok).split('/')[1] == 'v' or str(tok).split('/')[0] =='被']
                if '被' in verb_li and verb_li.index('被') < len(verb_li)-1:
                    verb_zh = verb_li[verb_li.index('被')+1]
                    # print(verb_zh)
                    print(e)
                    e += 1
                    print(en)
                    print(zh)
                    print(''.join(passive_verb[0])+':被'+verb_zh)
                    print('-----------')



                    with open(save_path,'a+') as t:
                        t.write(en+'\n'+zh+'\n'+''.join(passive_verb[0])+':被'+verb_zh+'\n')


    def word_literal_verb(self,pos_en='VERB',pos_zh='v'):
        e = 1
        
        
        for en,zh in zip(self.en_li[14000:], self.zh_li[14000:]):
          print(e)
          e += 1
          en_tok_li_pos = [str(en_pos.lemma_) for en_pos in nlp_en(en) if en_pos.pos_ == pos_en and en_pos.lemma_ == en_pos.text]
          
          zh_tok_li_pos = [str(zh_pos).split('/')[0] for zh_pos in posseg.cut(zh) if str(zh_pos).split('/')[1]==pos_zh]

          
          align_pair_li = []
          # all Chineses words comparison
          en2zh_li_pos = dict()
          for en_tok_pos in en_tok_li_pos:
              en_tokenn = tokenizer_en2zh(en_tok_pos, return_tensors="pt")
              generated_token = model.generate(**en_tokenn, forced_bos_token_id=tokenizer_en2zh.lang_code_to_id["zh_CN"])
              zh_text = tokenizer_en2zh.batch_decode(generated_token, skip_special_tokens=True)[0]
              en2zh_li_pos[zh_text] = en_tok_pos
          if len(list(set(en2zh_li_pos.keys()).intersection(set(zh_tok_li_pos)))) > 0:
              for ele in list(set(en2zh_li_pos.keys()).intersection(set(zh_tok_li_pos))):
                  
                  align_pair1 = en2zh_li_pos[ele]+':'+ele
                  print('--------en2zh')
                  print(en)
                  print(zh)
                  print(align_pair1)
                  align_pair_li.append(align_pair1)
          elif len(list(set(en2zh_li_pos.keys()).intersection(set(zh_tok_li_pos)))) == 0:
              for en2zh_word in en2zh_li_pos.keys():
                  for zh_word in zh_tok_li_pos:
                      if len(en2zh_word) > 0 and len(zh_word) > 0:
                          r = synonyms.compare(en2zh_word, zh_word, seg=True)
                          print(r)
                          if r>=0.8:
                              align_pair2 = en2zh_li_pos[en2zh_word] + ':' + zh_word
                              print('--------synonyms API')
                              print(en)
                              print(zh)
                              print(align_pair2)
                              align_pair_li.append(align_pair2)

          
          

          # all English word comparsion
          zh2en_li_pos = dict()
          for zh_tok_pos in zh_tok_li_pos:
              zh_tokenn = tokenizer_zh2en(zh_tok_pos, return_tensors="pt")
              generated_token_zh = model.generate(**zh_tokenn, forced_bos_token_id=tokenizer_zh2en.lang_code_to_id["en_XX"])
              en_text = tokenizer_zh2en.batch_decode(generated_token_zh, skip_special_tokens=True)[0].lower()
              zh2en_li_pos[en_text] = zh_tok_pos
          

          if len(list(set(zh2en_li_pos.keys()).intersection(set(en_tok_li_pos)))) > 0:
              for ele in list(set(zh2en_li_pos.keys()).intersection(set(en_tok_li_pos))):
                  
                  align_pair3 = ele+':'+zh2en_li_pos[ele]
                  align_pair_li.append(align_pair3)
                  print('-------zh2en')
                  print(en)
                  print(zh)
                  print(align_pair3)
                  
          
          elif len(list(set(zh2en_li_pos.keys()).intersection(set(en_tok_li_pos)))) == 0 :
              for zh2en_tok in zh2en_li_pos.keys():
                  if len(zh2en_tok) == 1:
                      zh2en_tok_syn_li = self.wordnet_synonym_en(wordnet.synsets(zh2en_tok))
                      for en_tok in en_tok_li_pos:
                          en_tok_syn_li = self.wordnet_synonym_en(wordnet.synsets(en_tok))
                          if zh2en_tok_syn_li.intersection(en_tok_syn_li) != ():
                              print('-------zh2en_wordnet')
                              align_pair4 = en_tok +':'+zh2en_li_pos[zh2en_tok]
                              print(en)
                              print(zh)
                              print(align_pair4)
                      
          for pair in list(set(align_pair_li)):
              with open(save_path,'a+') as y:
                  y.write(en+'\n'+zh+'\n'+pair+'\n')





    def bert_synonym_zh(self,tok):
        tokens = tokenizer_zh.encode(tok, add_special_tokens=True)
        input_ids = torch.tensor([tokens])
        with torch.no_grad():
            outputs = model_bert(input_ids)
        # Get the embeddings for the input word (ignoring the special tokens)
        word_embedding = outputs[0][0, 1:-1].mean(dim=0)




    def wordnet_synonym_en(synsets):
        lemmas = set()
        for synset in synsets:
            for lemma in synset.lemmas():
                lemmas.add(lemma.name().replace('_',' '))
        return lemmas
    


                      
                      
                                



          
            
            

    


            




spath = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/Bilingual/Education/Bi-Education.txt'
save_path = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature0_literal_translation/word_verb.txt'             
        
ptcl = Literal_translation(spath, save_path)

ptcl.word_literal_verb(pos_en='VERB',pos_zh='v')
